In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from selenium import webdriver
import time
import numpy as np
import ast
from datetime import datetime
from multiprocessing import Pool

In [2]:
df = pd.read_csv('../data/car_links_old.csv')
df.head()

,car_links,collected_at
0,https://www.hasznaltauto.hu/szemelyauto/volksw...,2023-06-26 12:00:00
1,https://www.hasznaltauto.hu/szemelyauto/citroe...,2023-06-26 12:00:00
2,https://www.hasznaltauto.hu/szemelyauto/merced...,2023-06-26 12:00:00
3,https://www.hasznaltauto.hu/szemelyauto/dacia/...,2023-06-26 12:00:00
4,https://www.hasznaltauto.hu/szemelyauto/merced...,2023-06-26 12:00:00


In [3]:
BASE_URL = 'https://www.hasznaltauto.hu/talalatilista/PCOG2VG3R3RDADH5S56ADN4Z3EEY7O2CQO2EEI5NGSXZLINGQROIZEVUDJAPZ6Z2NVQZVCVHLLDY47L4OJJJB43XPEGXEOUTIJB4JCV4QIGTMYWJ3BKIU4CBRPTAY5NECZNEQAJGF7OPJ6CB7EJCHBQGJIEBPMO4ZJE57JDB3CAC7LDSVHEUICRRZ3APOMHTEIC5NPWIZZWZ4JOSWHDXZHXKYDCOECG62ZWR6KNG5X3U3CQKHS4J2AVDYKJYEWO2CQGLSJZZT24ULD7GQCI3F5FE7W7EXQP4FCUIQMIW64YWGWLIAXDQRRZ3Z2F646IDCZVF2UUPJUP5QYHBEF4F7FVX6EMGUE2DEIYGPHUTR3FK74JF7QHH575DDQF7IQIW76QW7VAOIOTNZZ7B5TEX3KISPM5DHHV4VNBSJ62XQX4OTSKSKTSNMZMXPZWU67NNG6MVJLEFBIV57HKRVZNCSE6RO4TZRHAFV6QVA7MLEQOJK2GMAPQZFKCOOU4KEGPZAQNRMHTCF3GAGHAIUNK4GF3B7IRL5VR5HM3UM3OH43FVFGP3TBL43LQHO6DOVUIXRSARGRNRAT4SFL5KWN3LXNUS2MXJ5JLY23C7GFGPXN4JDYSFC67YRZMNAO7LGA52BJ663Y4YLCPMXXYFO5QBXEJO3CWBM7UWVQGQNTCQL2BFKNDIF6SBG2NOXK3AFJ4XNQQ4FUGDLRGCCXBUXNOA74OXOSO3IV67THQRSRMDHITEXBACWYDLEQ3JQ2U5TNCORXQIR5UUMZT53WUXS4OHTUPEZ3GTBR2KMA4I3CBW4TUEBHVKEH7NN7KJ7DUQ2UNWXQ3MUZVLUZPQCT2DO7DFB3GJK5QOZAHHVUJZTLP5Q53LKRJXEB57WD3X72QGI/page'


In [4]:
def get_links(page_number):
    url = BASE_URL+str(page_number)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    matches = soup.find_all('a', {'class':''})
    
    # Extract the hrefs from matches
    hrefs = pd.Series([match['href'] for match in matches if match.has_attr('href')])
    
    # Keep only those that a like for an car
    hrefs = hrefs[hrefs.str.contains('www.hasznaltauto.hu/szemelyauto', regex=False)].to_list()
    
    return hrefs


In [5]:
# Scrape all the links for cars from hasznaltauto.hu
# all_hrefs = [get_links(x) for x in range(1,296)]


In [8]:
"""
df_hrefs = pd.DataFrame({'car_links':np.concatenate(all_hrefs)})
df_hrefs['car_links'] = df_hrefs['car_links'].str.replace('#sid.*', '')
df_hrefs.drop_duplicates(inplace=True)
df_hrefs.reset_index(drop=True, inplace=True)
df_hrefs.to_csv('car_links.csv', index=False)
"""
df_hrefs = pd.read_csv('../data/car_links_old.csv')
df_hrefs.head()#"""

,car_links,collected_at
0,https://www.hasznaltauto.hu/szemelyauto/volksw...,2023-06-26 12:00:00
1,https://www.hasznaltauto.hu/szemelyauto/citroe...,2023-06-26 12:00:00
2,https://www.hasznaltauto.hu/szemelyauto/merced...,2023-06-26 12:00:00
3,https://www.hasznaltauto.hu/szemelyauto/dacia/...,2023-06-26 12:00:00
4,https://www.hasznaltauto.hu/szemelyauto/merced...,2023-06-26 12:00:00


In [9]:
link = df['car_links'].values[0]

In [28]:
def scrape_car_data(link):
    """
    Scrape car data from a link
    :param link: link of the car in hasznaltauto.hu
    :return: tuple of:
        - advertisement data which is a dictionary
        with well-defined keys as it contains necessary information about the vehicle.
        - special information about the given car it can be anything without a well-defined stricture.
    """
    # Get the html content of the link
    response = requests.get(link)

    # Parse the html content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Get the data from the base advertisement information table
    advertisement_data = soup.find_all("table", {"class": "hirdetesadatok"})

    # If there is no advertisement_data that means the car is not available anymore
    if len(advertisement_data) == 0:
        return None, None

    # Get the table in html format
    table_html = str(soup.find_all('table', {'class': 'hirdetesadatok'})[0])

    # Read the table with pandas and
    advertisement_data = pd.read_html(table_html)[0]

    # Clean the data
    advertisement_data.columns = ['key', 'value']
    # Remove the rows where the key is not contains ':' at the end
    # This is because the table contains some other information which is not irrelevant,
    # not real key value pairs
    advertisement_data = advertisement_data[advertisement_data['key'].str.contains(':$', regex=True)]

    # Transform dataframe key to columns and values to rows
    # This will be easier to work with as we collect the car data inta rows of a dataframe
    advertisement_data = advertisement_data[advertisement_data['key'].str.contains(':$', regex=True)]
    advertisement_data = advertisement_data.T
    advertisement_data.reset_index(drop=True, inplace=True)
    advertisement_data.columns = advertisement_data.iloc[0]
    advertisement_data = advertisement_data[1:]

    # Seller information
    h4_headers = [x.text for x in soup.find_all("h4")]
    # Decide if the seller is a shop or private person
    advertisement_data["buy_from_shop"] = 'Kereskedés adatai' in h4_headers

    # Sale contact
    contacts = soup.find_all('span', {"class": "contact-button-text"})
    for i in range(len(contacts)):
        advertisement_data[f"content_info_{i}"] = contacts[i].text

    # Get equipment info
    equipments = soup.find_all("div", {"class": "row felszereltseg"})
    if equipments:
        equipments = equipments[0].text.split("\n")

    # Get other info
    other = soup.find_all("div", {"class": "egyebinformacio"})
    if other:
        other = other[0].text.split("\n")

    # Get description
    description = soup.find_all("div", {"class": "leiras"})
    if description:
        description = [description[0].text]

    # Get oll special info about the car and clean it
    special_car_info = equipments + other + description
    if special_car_info:
        special_car_info = pd.Series(special_car_info)
        special_car_info = special_car_info.str.strip()
        special_car_info = special_car_info.str.lower()
        special_car_info = special_car_info.dropna()
        special_car_info = special_car_info[~special_car_info.isin(["", "-"])]

    return advertisement_data, special_car_info


In [17]:
rename_dict = {
    'vételár': 'price (HUF)',
    'állapot': 'state',
    'kivitel': 'desing',
    'járműelőélet': 'vehicle history',
    'finanszírozás': 'financing',
    'km. óra állás': 'clock position (km)',
    'szállítható szem. száma': 'shippable persons number',
    'ajtók száma': 'number of doors',
    'szín': 'color',
    'saját tömeg': 'own weight (kg)',
    'teljes tömeg': 'total weight (kg)',
    'csomagtartó': 'trunk (l)',
    'klíma fajtája': 'type of climate',
    'tető': 'roof',
    'üzemanyag': 'fuel',
    'hengerűrtartalom': 'cylinder capacity (cm3)',
    'teljesítmény': 'performance (kW)',
    'henger-elrendezés': 'cylinder arrangement',
    'hajtás': 'drive',
    'sebességváltó': 'gearbox',
    'okmányok jellege': 'nature of documents',
    'műszaki vizsga érvényes': 'MOT is valid (days)',
    'kárpit színe (1)': 'upholstery color (1)',
    'kárpit színe (2)': 'upholstery color (2)',
    'kezdőrészlet': 'initial part',
    'futamidő': 'term',
    'link': 'link',
    'age_days': 'age_days',
    'summer tire width': 'summer tire width',
    'summer tires apect ratio': 'summer tires aspect ratio',
    'summer tires construction type': 'summer tires construction type',
    'summer tires rim diameter': 'summer tires rim diameter',
    'winter tire width': 'winter tire width',
    'winter tires apect ratio': 'winter tires aspect ratio',
    'winter tires construction type': 'winter tires construction type',
    'winter tires rim diameter': 'winter tires rim diameter',
    'back summer tire width': 'back summer tire width',
    'back summer tires apect ratio': 'back summer tires aspect ratio',
    'back summer tires construction type': 'back summer tires construction type',
    'back summer tires rim diameter': 'back summer tires rim diameter',
    'back winter tire width': 'back winter tire width',
    'back winter tires apect ratio': 'back winter tires aspect ratio',
    'back winter tires construction type': 'back winter tires construction type',
    'back winter tires rim diameter': 'back winter tires rim diameter',
    'speaker count': 'speaker count',
    'elektromos ablak elöl': 'electric window in front',
    'vezetőoldali légzsák': 'abs (anti-lock braking)',
    'abs (blokkolásgátló)': 'driver airbag',
    'utasoldali légzsák': 'passenger airbag',
    'elektromos tükör': 'electric mirror',
    'centrálzár': 'central locking',
    'állítható kormány': 'predictable government',
    'oldallégzsák': 'side airbag',
    'ülésmagasság állítás': 'seat height adjustment',
    'szervokormány': 'power steering',
    'könnyűfém felni': 'esp (speed stabilizer)',
    'esp (menetstabilizátor)': 'alloy wheels',
    'elektromos ablak hátul': 'electric rear window',
    'asr (kipörgésgátló)': 'asr (traction control)',
    'fedélzeti komputer': 'board computer',
    'fűthető tükör': 'heatable mirror',
    'isofix rendszer': 'isofix system',
    'hátsó fejtámlák': 'rear head restraints',
    'indításgátló (immobiliser)': 'immobiliser',
    'rendszeresen karbantartott': 'regularly maintained',
    'színezett üveg': 'stained glass',
    'függönylégzsák': 'curtain airbag',
    'tempomat': 'cruise control',
    'ködlámpa': 'fog lamp',
    'középső kartámasz': 'middle armrest',
    'nem dohányzó': 'non-smoking',
    'multifunkciós kormánykerék': 'multifunction steering wheel',
    'garantált km futás': 'guaranteed mileage',
    'bőrkormány': 'leather steering wheel',
    'azonnal elvihető': 'immediately available',
    'tolatóradar': 'reverse radar',
    'bluetooth-os kihangosító': 'bluetooth speakerphone',
    'kikapcsolható légzsák': 'disabled airbag',
    '2 din': '2 din',
    '20%-tól elvihető': 'Take away from 20%',
    '230v csatlakozó hátul': '230v socket at the back',
    '30%-tól elvihető': 'Can be taken away from 30%',
    '360 fokos kamerarendszer': '360 degree camera system',
    'ads (adaptív lengéscsillapító)': 'ads (adaptive shock absorber)',
    'alcantara kárpit': 'alcantara upholstery',
    'android auto': 'android auto',
    'apple carplay': 'apple carplay',
    'ard (automatikus távolságtartó)': 'ard (automatic spacer)',
    'automata fényszórókapcsolás': 'automatic headlight switching',
    'automata távfény': 'automatic high beam',
    'automatikus segélyhívó': 'automatic emergency call',
    'automatikusan sötétedő belső tükör': 'automatically dimming interior mirror',
    'automatikusan sötétedő külső tükör': 'automatically dimming exterior mirror',
    'autóbeszámítás lehetséges': 'car offsetting is possible',
    'autótelefon': 'car phone',
    'aux csatlakozó': 'aux connector',
    'bekanyarodási segédfény': 'turning light',
    'bi-xenon fényszóró': 'bi-xenon headlight',
    'bőr belső': 'leather interior',
    'bőr-szövet huzat': 'skin-fabric cover',
    'cd-s autórádió': 'cd car radio',
    'csomag rögzítő': 'package fixer',
    'defektjavító készlet': 'defect repair kit',
    'defekttűrő abroncsok': 'flat tires',
    'deréktámasz': 'lumbar support',
    'digitális műszeregység': 'digital instrument unit',
    'dvd': 'DVD',
    'dönthető utasülések': 'reclinable passenger seats',
    'ebd/ebv (elektronikus fékerő-elosztó)': 'ebd/ebv (electronic brake force distribution)',
    'edc (elektronikus lengéscsillapítás vezérlés)': 'edc (electronic damping control)',
    'eds (elektronikus differenciálzár)': 'eds (electronic differential lock)',
    'elektromos csomagtérajtó-mozgatás': 'electric tailgate movement',
    'elektromos ülésállítás utasoldal': 'electric seat adjustment passenger side',
    'elektromos ülésállítás vezetőoldal': 'electric seat adjustment driver side',
    'elektromosan behajtható külső tükrök': 'electrically folding exterior mirrors',
    'elektronikus futómű hangolás': 'electronic chassis tuning',
    'elektronikus rögzítőfék': 'electronic parking brake',
    'első forgalomba helyezés magyarországon': 'first placing on the market in Hungary',
    'első tulajdonostól': 'from the first owner',
    'első-hátsó parkolóradar': 'front-rear parking radar',
    'esőszenzor': 'rain sensor',
    'faberakás': 'timbering',
    'frissen szervizelt': 'freshly serviced',
    'full extra': 'full extra',
    'fáradtságérzékelő': 'fatigue sensor',
    'fékasszisztens': 'brake assistant',
    'fényszóró magasságállítás': 'headlight height adjustment',
    'fényszórómosó': 'headlight washer',
    'fűthető ablakmosó fúvókák': 'heatable window washer nozzles',
    'fűthető első és hátsó ülések': 'heated front and rear seats',
    'fűthető első ülés': 'heated front seat',
    'fűthető kormány': 'heated steering wheel',
    'fűtőszálas szélvédő': 'heated fiber windshield',
    'garanciális': 'guaranteed',
    'garázsban tartott': 'kept in garage',
    'gps (navigáció)': 'gps (navigation)',
    'gps nyomkövető': 'gps tracker',
    'guminyomás-ellenőrző rendszer': 'tire pressure monitoring system',
    'gyalogos légzsák': 'pedestrian airbag',
    'gyári erősítő': 'factory amplifier',
    'hangvezérlés': 'voice control',
    'hifi': 'hifi',
    'holttér-figyelő rendszer': 'blind spot monitoring system',
    'hud / head-up display': 'hud / head-up display',
    'hátsó keresztirányú forgalomra figyelmeztetés': 'rear cross traffic warning',
    'hátsó oldal légzsák': 'rear side airbag',
    'hölgy tulajdonostól': 'from lady owner',
    'hűthető kartámasz': 'coolable armrest',
    'hűthető kesztyűtartó': 'refrigerated glove box',
    'iphone/ipod csatlakozó': 'iphone/ipod connector',
    'kanyarkövető fényszóró': 'cornering headlight',
    'keveset futott': 'ran little',
    'kiegészítő fényszóró': 'additional headlight',
    'kihangosító': 'hands-free',
    'koccanásgátló': 'anti-scratch',
    'kormányról vezérelhető hifi': 'steering wheel controllable hi-fi',
    'kormányváltó': 'steering gear',
    'kulcsnélküli indítás': 'keyless start',
    'kulcsnélküli nyitórendszer': 'keyless entry system',
    'középső légzsák elöl': 'front central airbag',
    'led fényszóró': 'led headlight',
    'led mátrix fényszóró': 'led matrix headlight',
    'lejtmenet asszisztens': 'descent assistant',
    'memóriakártya-olvasó': 'memory card reader',
    'memóriás utasülés': 'memory passenger seat',
    'memóriás vezetőülés': 'memory driver seat',
    'menetfény': 'running light',
    'motorbeszámítás lehetséges': 'engine calculation possible',
    'mp3 lejátszás': 'play mp3',
    'mp4 lejátszás': 'play mp4',
    'msr (motorféknyomaték szabályzás)': 'msr (motor brake torque regulation)',
    'multifunkcionális kijelző': 'multifunctional display',
    'második tulajdonostól': 'from second owner',
    'mélynyomó': 'subwoofer',
    'parkolóasszisztens': 'parking assistant',
    'plüss kárpit': 'plush upholstery',
    'pótkerék': 'spare wheel',
    'radaros fékasszisztens': 'radar brake assistant',
    'riasztó': 'alarm',
    'rádió': 'radio',
    'rádiós magnó': 'radio tape recorder',
    'részecskeszűrő': 'particulate filter',
    'sebességfüggő szervókormány': 'speed-dependent power steering',
    'sportfutómű': 'sports chassis',
    'sportülések': 'sports seats',
    'start-stop/motormegállító rendszer': 'start-stop/engine stop system',
    'sávtartó rendszer': 'lane keeping system',
    'sávváltó asszisztens': 'lane change assistant',
    'tetőcsomagtartó': 'roof rack',
    'tolatókamera': 'reversing camera',
    'tolótető (napfénytető)': 'sliding roof (sunroof)',
    'tolótető - elektromos': 'sliding roof - electric',
    'tábla-felismerő funkció': 'plate recognition function',
    'távolsági fényszóró asszisztens': 'high beam assistant',
    'távolságtartó tempomat': 'distance control cruise control',
    'térdlégzsák': 'knee airbag',
    'törzskönyv': 'herdbook',
    'usb csatlakozó': 'USB connector',
    'vezetett szervizkönyv': 'guided service book',
    'vezeték nélküli telefontöltés': 'wireless phone charging',
    'visszagurulás-gátló': 'anti-rollback',
    'vonóhorog': 'tow hook',
    'vonóhorog - levehető fejjel': 'tow hook - with removable head',
    'végig vezetett szervizkönyv': 'service book kept to the end',
    'vészfék asszisztens': 'emergency brake assistant',
    'wifi hotspot': 'wifi hotspot',
    'wma lejátszás': 'play wma',
    'xenon fényszóró': 'xenon headlight',
    'áfa visszaigényelhető': 'VAT can be reclaimed',
    'állítható combtámasz': 'adjustable thigh support',
    'állítható felfüggesztés': 'adjustable suspension',
    'állítható hátsó ülések': 'adjustable rear seats',
    'álló helyzeti klíma': 'stationary climate',
    'állófűtés': 'stationary heating',
    'érintőkijelző': 'touch screen',
    'üléshűtés/szellőztetés': 'seat cooling/ventilation',
    'ütközés veszélyre felkészítő rendszer': 'collision hazard preparation system',
    '25%-tól elvihető': 'Take away from 25%',
    'bekanyarodási asszisztens': 'turning assistant',
    'cd tár': 'cd library',
    'elektromosan állítható fejtámlák': 'electrically adjustable headrests',
    'erősítő': 'amplifier',
    'otthoni hálózati töltő': 'home mains charger',
    'rablásgátló': 'anti-theft',
    'type2 töltőkábel': 'type2 charging cable',
    'vonóhorog - elektromosan kihajtható': 'towing hook - electrically foldable'
}

In [16]:
# Get the html content of the link
response = requests.get(link)

# Parse the html content
soup = BeautifulSoup(response.content, 'html.parser')

# Get the data from the base advertisement information table
advertisement_data = soup.find_all("table", {"class": "hirdetesadatok"})

# Get the table in html format
table_html = str(soup.find_all('table', {'class': 'hirdetesadatok'})[0])

# Read the table with pandas and
advertisement_data = pd.read_html(table_html)[0]

# Clean the data
advertisement_data.columns = ['key', 'value']

advertisement_data = advertisement_data[advertisement_data['key'].str.contains(':$', regex=True)]
advertisement_data = dict(zip(advertisement_data['key'], advertisement_data['value']))


advertisement_data

{'Vételár:': '6\xa0895\xa0000\xa0Ft',
 'Vételár EUR:': '€\xa018\xa0604',
 'Évjárat:': '2016/3',
 'Állapot:': 'Kitűnő',
 'Kivitel:': 'Sedan',
 'Járműelőélet:': 'Elérhető autó-előélet alvázszám alapján',
 'Finanszírozás:': '25%-tól elvihető',
 'Km. óra állás:': '110\xa0000\xa0km',
 'Szállítható szem. száma:': '5 fő',
 'Ajtók száma:': '4',
 'Szín:': 'Fekete (metál)',
 'Saját tömeg:': '1\xa0426 kg',
 'Teljes tömeg:': '2\xa0040 kg',
 'Csomagtartó:': '586 liter',
 'Klíma fajtája:': 'Digitális többzónás klíma',
 'Tető:': 'Lemeztető',
 'Üzemanyag:': 'Dízel',
 'Hengerűrtartalom:': '1\xa0968 cm³',
 'Teljesítmény:': '110\xa0kW, 150\xa0LE',
 'Henger-elrendezés:': 'Soros',
 'Hajtás:': 'Első kerék',
 'Sebességváltó:': 'Automata (6 fokozatú)',
 'Okmányok jellege:': 'Érvényes magyar okmányokkal',
 'Műszaki vizsga érvényes:': '2025/5',
 'Nyári gumi méret:': '215/60\xa0R\xa016'}

In [22]:
translated_dict = {}
for k in advertisement_data.keys():
    try:
        norm_k = k.lower().replace(':', '')
        translation = rename_dict[norm_k]
        translated_dict[f"{norm_k} ({translation})"] = advertisement_data[k]
    except:
        print('not', k)
    
translated_dict

not Vételár EUR:
not Évjárat:
not Nyári gumi méret:


{'vételár (price (HUF))': '6\xa0895\xa0000\xa0Ft',
 'állapot (state)': 'Kitűnő',
 'kivitel (desing)': 'Sedan',
 'járműelőélet (vehicle history)': 'Elérhető autó-előélet alvázszám alapján',
 'finanszírozás (financing)': '25%-tól elvihető',
 'km. óra állás (clock position (km))': '110\xa0000\xa0km',
 'szállítható szem. száma (shippable persons number)': '5 fő',
 'ajtók száma (number of doors)': '4',
 'szín (color)': 'Fekete (metál)',
 'saját tömeg (own weight (kg))': '1\xa0426 kg',
 'teljes tömeg (total weight (kg))': '2\xa0040 kg',
 'csomagtartó (trunk (l))': '586 liter',
 'klíma fajtája (type of climate)': 'Digitális többzónás klíma',
 'tető (roof)': 'Lemeztető',
 'üzemanyag (fuel)': 'Dízel',
 'hengerűrtartalom (cylinder capacity (cm3))': '1\xa0968 cm³',
 'teljesítmény (performance (kW))': '110\xa0kW, 150\xa0LE',
 'henger-elrendezés (cylinder arrangement)': 'Soros',
 'hajtás (drive)': 'Első kerék',
 'sebességváltó (gearbox)': 'Automata (6 fokozatú)',
 'okmányok jellege (nature of docum

In [20]:
translated_dict.keys()

dict_keys([])

In [14]:
link

'https://www.hasznaltauto.hu/szemelyauto/volkswagen/passat/volkswagen_passat_2.0_tdi_bmt_comfortline_dsg_elso_tulajdonostol_gyari_fenyezesu_garantalt_kilometerrel-19076469'

In [15]:
# Get the html content of the link
response = requests.get(link)

# Parse the html content
soup = BeautifulSoup(response.content, 'html.parser')

# Get the data from the base advertisement information table
advertisement_data = soup.find_all("table", {"class": "hirdetesadatok"})

# Get the table in html format
table_html = str(soup.find_all('table', {'class': 'hirdetesadatok'})[0])

# Read the table with pandas and
advertisement_data = pd.read_html(table_html)[0]

# Clean the data
advertisement_data.columns = ['key', 'value']
# Remove the rows where the key is not contains ':' at the end
# This is because the table contains some other information which is not irrelevant,
# not real key value pairs
advertisement_data = advertisement_data[advertisement_data['key'].str.contains(':$', regex=True)]
advertisement_data = advertisement_data.T
advertisement_data.reset_index(drop=True, inplace=True)
advertisement_data.columns = advertisement_data.iloc[0]
advertisement_data = advertisement_data[1:]
advertisement_data


,Vételár:,Vételár EUR:,Évjárat:,Állapot:,Kivitel:,Járműelőélet:,Finanszírozás:,Km. óra állás:,Szállítható szem. száma:,Ajtók száma:,...,Tető:,Üzemanyag:,Hengerűrtartalom:,Teljesítmény:,Henger-elrendezés:,Hajtás:,Sebességváltó:,Okmányok jellege:,Műszaki vizsga érvényes:,Nyári gumi méret:
1,6 895 000 Ft,€ 18 604,2016/3,Kitűnő,Sedan,Elérhető autó-előélet alvázszám alapján,25%-tól elvihető,110 000 km,5 fő,4,...,Lemeztető,Dízel,1 968 cm³,"110 kW, 150 LE",Soros,Első kerék,Automata (6 fokozatú),Érvényes magyar okmányokkal,2025/5,215/60 R 16


In [ ]:
advertisement_data

In [7]:
advertisement_data = advertisement_data.T
advertisement_data.reset_index(drop=True, inplace=True)
advertisement_data.columns = advertisement_data.iloc[0]
advertisement_data = advertisement_data[1:]
advertisement_data


In [52]:
link = all_links[0]
link

'https://www.hasznaltauto.hu/szemelyauto/volkswagen/passat/volkswagen_passat_2.0_tdi_bmt_comfortline_dsg_elso_tulajdonostol_gyari_fenyezesu_garantalt_kilometerrel-19076469'

In [66]:
response = requests.get(link)
soup = BeautifulSoup(response.content, 'html.parser')

header = soup.find_all('div', {'class':'data-relay-banner'})

header = ast.literal_eval(header[0]['data-vehicle-information-banner-parameters'])
advertisement_data =  soup.find_all('table', {'class':'hirdetesadatok'})
# Get the text from xml
advertisement_data = advertisement_data[0].get_text(separator='#')

# Clean it
advertisement_data = advertisement_data.strip()
advertisement_data = re.sub(r'#\n', '', advertisement_data)

# Get lines
advertisement_data = advertisement_data.split('#')
advertisement_data

['Ár, költségek',
 'Vételár:',
 '6\xa0895\xa0000\xa0Ft',
 'Vételár EUR:',
 '€\xa018\xa0950                        Átírás kalkulátor                                HIRDETÉS        ',
 'Általános adatok',
 'Évjárat:',
 '2016/3',
 'Állapot:',
 'Kitűnő',
 'Kivitel:',
 'Sedan',
 'Járműelőélet:',
 'Elérhető autó-előélet alvázszám alapján                        Ellenőrzöm                    ',
 'Finanszírozás',
 'Finanszírozás:',
 '25%-tól elvihető',
 'Jármű adatok',
 'Km. óra állás:',
 '110\xa0000\xa0km',
 'Szállítható szem. száma:',
 '5 fő',
 'Ajtók száma:',
 '4',
 'Szín:',
 'Fekete (metál)',
 'Saját tömeg:',
 '1\xa0426 kg',
 'Teljes tömeg:',
 '2\xa0040 kg',
 'Csomagtartó:',
 '586 liter',
 'Klíma fajtája:',
 'Digitális többzónás klíma',
 'Tető:',
 'Lemeztető',
 'Motor adatok',
 'Üzemanyag:',
 'Dízel',
 'Hengerűrtartalom:',
 '1\xa0968 cm³',
 'Teljesítmény:',
 '110\xa0kW, 150\xa0LE',
 'Henger-elrendezés:',
 'Soros',
 'Hajtás:',
 'Első kerék',
 'Sebességváltó:',
 'Automata (6 fokozatú)',
 'Okm

In [63]:
# Clean lines and create pairs
advertisement_data = pd.Series(advertisement_data)
        
# Add header to advertisment data
advertisement_data.update(header)
advertisement_data

0                                         Ár, költségek
1                                              Vételár:
2                                          6 895 000 Ft
3                                          Vételár EUR:
4     € 18 950                        Átírás kalkulá...
5                                      Általános adatok
6                                              Évjárat:
7                                                2016/3
8                                              Állapot:
9                                                Kitűnő
10                                             Kivitel:
11                                                Sedan
12                                        Járműelőélet:
13    Elérhető autó-előélet alvázszám alapján       ...
14                                        Finanszírozás
15                                       Finanszírozás:
16                                     25%-tól elvihető
17                                         Jármű

In [64]:
advertisement_data.update(header)

In [65]:
advertisement_data

0                                         Ár, költségek
1                                              Vételár:
2                                          6 895 000 Ft
3                                          Vételár EUR:
4     € 18 950                        Átírás kalkulá...
5                                      Általános adatok
6                                              Évjárat:
7                                                2016/3
8                                              Állapot:
9                                                Kitűnő
10                                             Kivitel:
11                                                Sedan
12                                        Járműelőélet:
13    Elérhető autó-előélet alvázszám alapján       ...
14                                        Finanszírozás
15                                       Finanszírozás:
16                                     25%-tól elvihető
17                                         Jármű

In [62]:


keys_id = advertisement_data.str.contains(':$')
values_id = np.where(keys_id)[0] + 1
keys = advertisement_data[keys_id]
values = advertisement_data[values_id]
advertisement_data = dict(zip(keys, values))

['Ár, költségek',
 'Vételár:',
 '6\xa0895\xa0000\xa0Ft',
 'Vételár EUR:',
 '€\xa018\xa0950                        Átírás kalkulátor                                HIRDETÉS        ',
 'Általános adatok',
 'Évjárat:',
 '2016/3',
 'Állapot:',
 'Kitűnő',
 'Kivitel:',
 'Sedan',
 'Járműelőélet:',
 'Elérhető autó-előélet alvázszám alapján                        Ellenőrzöm                    ',
 'Finanszírozás',
 'Finanszírozás:',
 '25%-tól elvihető',
 'Jármű adatok',
 'Km. óra állás:',
 '110\xa0000\xa0km',
 'Szállítható szem. száma:',
 '5 fő',
 'Ajtók száma:',
 '4',
 'Szín:',
 'Fekete (metál)',
 'Saját tömeg:',
 '1\xa0426 kg',
 'Teljes tömeg:',
 '2\xa0040 kg',
 'Csomagtartó:',
 '586 liter',
 'Klíma fajtája:',
 'Digitális többzónás klíma',
 'Tető:',
 'Lemeztető',
 'Motor adatok',
 'Üzemanyag:',
 'Dízel',
 'Hengerűrtartalom:',
 '1\xa0968 cm³',
 'Teljesítmény:',
 '110\xa0kW, 150\xa0LE',
 'Henger-elrendezés:',
 'Soros',
 'Hajtás:',
 'Első kerék',
 'Sebességváltó:',
 'Automata (6 fokozatú)',
 'Okm

In [54]:
response = requests.get(link)
soup = BeautifulSoup(response.content, "html.parser")

header = soup.find_all("div", {"class": "data-relay-banner"}, )

# Get the header information
# It is in a dictionary format but as a string, so we need to convert it
header = ast.literal_eval(
    header[0]["data-vehicle-information-banner-parameters"]
)

# Get the data from the base advertisement information table
advertisement_data = soup.find_all("table", {"class": "hirdetesadatok"})


In [56]:
header

{'vetelar': 6895000,
 'gyartasiev': 2016,
 'gyartasiho': 3,
 'gepjarmutipus': 1,
 'gepjarmumarka': 'VOLKSWAGEN PASSAT 2.0 TDI BMT Comfortline DSG ELSŐ TULAJDONOSTÓL. GYÁRI FÉNYEZÉSŰ. GARANTÁLT KILÓMÉTERREL!!!',
 'kalk_forras': 'HAHU',
 'cofidisPartnerId': 17987,
 'adId': 19076469,
 'adURL': 'https:\\/\\/www.hasznaltauto.hu\\/szemelyauto\\/volkswagen\\/passat\\/volkswagen_passat_2.0_tdi_bmt_comfortline_dsg_elso_tulajdonostol_gyari_fenyezesu_garantalt_kilometerrel-19076469'}

In [50]:
is_link_duplicated = all_links.duplicated()

In [51]:
is_link_duplicated.sum()

0

In [48]:
# Update dataset with new cars
def load_existing_links(filename):
    df = pd.read_csv(filename)
    return df['car_links'].values

def get_all_links():
    all_links = []
    page_number = 1
    while True:
        links_on_page = get_links(page_number)
        if not links_on_page:
            break
        page_number += 1
        all_links.append(links_on_page)
        
    all_links =  pd.Series(np.concatenate(all_links))
    all_links = all_links.str.replace('#sid.*', '')
    return all_links


def get_new_links(existing_links):
    all_hrefs = get_all_links()

    # Extract the hrefs from page results
    new_links = all_hrefs[~all_hrefs.isin(existing_links)]
    new_links = new_links.drop_duplicates()
    
    return new_links


def update_links(filename):
    existing_links = load_existing_links(filename)
    new_links = get_new_links(existing_links)

    # Add new links to existing ones and save to csv
    all_links = list(existing_links) + list(new_links)
    df_hrefs = pd.DataFrame({'car_links': all_links})
    df_hrefs.drop_duplicates(inplace=True)
    df_hrefs.reset_index(drop=True, inplace=True)
    df_hrefs.to_csv(filename, index=False)

    return new_links

# Use the function like this:
new_links = update_links('car_links.csv')


KeyboardInterrupt: 

In [72]:
new_links = new_links[~new_links.isin(all_links)]

In [82]:
%%time
for i, link in enumerate(new_links):
    advertisement_data, special_car_info = scrape_car_data(link)
    res_list.append([advertisement_data, special_car_info, link])
    
    if (i % 100) == 0:
        print(i)
        with open('hasnaltauto_scraped.pickle', 'wb') as file:
            pickle.dump(res_list, file)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
CPU times: user 4min 33s, sys: 42.4 s, total: 5min 15s
Wall time: 21min 18s


In [66]:
i

1439